### Helper Scripts for programming automated testing for a 32-bit ALU on the FPGA

Below are the scripts used to generate the repeated portions of the alu_auto_tester module:

In [99]:
def instantiate_states(no_states):
    tests = ""
    
    for i in range(no_states):
        tests += f", TEST_{i}"
    
    return "enum AutoStates {STANDBY" + tests + "}"

In [101]:
instantiate_states(1)

'enum AutoStates {STANDBY, TEST_0}'

In [103]:
def instantiate_tester (test_no, input_a, input_b, opcode, expected):
    return f"            alu_statement_tester test_{test_no} (#INPUT_A(32h{input_a}), #INPUT_B(32h{input_b}), #ALUCODE(6b{opcode}), #EXPECTED(32h{expected}))"

In [105]:
def state_case(state_no, is_final):
    output = ""
    output += f"            AutoStates.TEST_{state_no}:\n"
    output += f"                io_segment = test_{state_no}.io_segment\n"
    output += f"                io_select = test_{state_no}.io_select\n"
    output += f"                alucode_led = test_{state_no}.alucode_led\n"
    output += f"                led = test_{state_no}.led\n"
    output += f"                test_{state_no}.start = 1\n"
    output += f"                if(test_{state_no}.done)\n"
    if (is_final):
        output += f"                    states.d = AutoStates.STANDBY"
    else:
        output += f"                    states.d = AutoStates.TEST_{state_no + 1}"
    
    return output

In [107]:
print(state_case(1,False))

            AutoStates.TEST_1:
                io_segment = test_1.io_segment
                io_select = test_1.io_select
                alucode_led = test_1.alucode_led
                led = test_1.led
                test_1.start = 1
                if(test_1.done)
                    states.d = AutoStates.TEST_2


### Test Cases
Test cases are in the format (a_input, b_input, opcode, expected_output):

In [109]:
test_cases = [
    ("abcd1234", "00000000", "000000", "abcd1234"),  # ADD: Zero Addition A + 0  
    ("00000000", "abcd1234", "000000", "abcd1234"),  # ADD: Zero Addition 0 + A  
    ("12345678", "23456789", "000000", "3579be01"),  # ADD: Positive Numbers  
    ("7fffffff", "7fffffff", "000000", "fffffffe"),  # ADD: Overflow  
    
    ("abcd1234", "00000000", "000001", "abcd1234"),  # SUB: Zero Subtraction A - 0  
    ("00000000", "abcd1234", "000001", "5432edcc"),  # SUB: Zero Subtraction 0 - A  
    ("00000000", "7fffffff", "000001", "80000001"),  # SUB: Boundary Value  
    
    ("abcd1234", "00000000", "000010", "00000000"),  # MUL: Zero Multiplication  
    ("abcd1234", "00000001", "000010", "abcd1234"),  # MUL: Multiplication by One  
    ("00001234", "00005678", "000010", "06260060"),  # MUL: Positive Numbers
    
    ("abcd1234", "00000000", "011000", "00000000"),  # AND: All Zeros  
    ("abcd1234", "ffffffff", "011000", "abcd1234"),  # AND: All Ones  
    ("abcd1234", "abcd1234", "011000", "abcd1234"),  # AND: Identity Check
    ("ffffffff", "00000000", "011000", "00000000"),  # AND: Complementary Check
    
    ("abcd1234", "00000000", "011110", "abcd1234"),  # OR: All Zeros  
    ("abcd1234", "ffffffff", "011110", "ffffffff"),  # OR: All Ones
    ("abcd1234", "abcd1234", "011110", "abcd1234"),  # OR: Identity Check
    
    ("abcd1234", "00000000", "010110", "abcd1234"),  # XOR: Identity Check  
    ("abcd1234", "abcd1234", "010110", "00000000"),  # XOR: Self-XOR
    ("ffffffff", "00000000", "010110", "ffffffff"),  # XOR: Complementary Check
    
    ("abcd1234", "00000000", "011010", "abcd1234"),  # LDR: Load Function  
    ("ef425678", "00000000", "011010", "ef425678"),  # LDR: Load Function
    ("cd426789", "00000000", "011010", "cd426789"),  # LDR: Load Function
    
    ("abcd1234", "00000000", "100000", "abcd1234"),  # SHL: Zero Shift  
    ("00000001", "0000001f", "100000", "80000000"),  # SHL: Maximum Shift  
    
    ("abcd1234", "00000000", "100001", "abcd1234"),  # SHR: Zero Shift  
    ("80000000", "0000001f", "100001", "00000001"),  # SHR: Maximum Shift
    
    ("abcd1234", "00000000", "100011", "abcd1234"),  # SRA: Zero Shift  
    ("80000000", "0000001f", "100011", "ffffffff"),  # SRA: Arithmetic Right Shift
    
    ("abcd1234", "abcd1234", "110011", "00000001"),  # CMPEQ: Equality  
    ("abcd1234", "ef421234", "110011", "00000000"),  # CMPEQ: Inequality  
    
    ("abcd1234", "ef421234", "110101", "00000001"),  # CMPLT: Less Than
    ("abcd1234", "abcd1234", "110101", "00000000"),  # CMPLT: Equal
    ("ef421234", "abcd1234", "110101", "00000000"),  # CMPLT: Greater Than

    ("abcd1234", "ef421234", "110101", "00000001"),  # CMPLE: Less Than
    ("abcd1234", "abcd1234", "110111", "00000001"),  # CMPLE: Equal  
    ("ef421234", "abcd1234", "110111", "00000000")  # CMPLE: Greater Than
]

Generate a list of states for alu_test_auto & instantiate the statement tester modules: 

In [111]:
l = len(test_cases)

print(instantiate_states(l))

for i in range(l):
    print(instantiate_tester(i, test_cases[i][0], test_cases[i][1], test_cases[i][2], test_cases[i][3]))

enum AutoStates {STANDBY, TEST_0, TEST_1, TEST_2, TEST_3, TEST_4, TEST_5, TEST_6, TEST_7, TEST_8, TEST_9, TEST_10, TEST_11, TEST_12, TEST_13, TEST_14, TEST_15, TEST_16, TEST_17, TEST_18, TEST_19, TEST_20, TEST_21, TEST_22, TEST_23, TEST_24, TEST_25, TEST_26, TEST_27, TEST_28, TEST_29, TEST_30, TEST_31, TEST_32, TEST_33, TEST_34, TEST_35, TEST_36}
            alu_statement_tester test_0 (#INPUT_A(32habcd1234), #INPUT_B(32h00000000), #ALUCODE(6b000000), #EXPECTED(32habcd1234))
            alu_statement_tester test_1 (#INPUT_A(32h00000000), #INPUT_B(32habcd1234), #ALUCODE(6b000000), #EXPECTED(32habcd1234))
            alu_statement_tester test_2 (#INPUT_A(32h12345678), #INPUT_B(32h23456789), #ALUCODE(6b000000), #EXPECTED(32h3579be01))
            alu_statement_tester test_3 (#INPUT_A(32h7fffffff), #INPUT_B(32h7fffffff), #ALUCODE(6b000000), #EXPECTED(32hfffffffe))
            alu_statement_tester test_4 (#INPUT_A(32habcd1234), #INPUT_B(32h00000000), #ALUCODE(6b000001), #EXPECTED(32habcd123

In [113]:
l = len(test_cases)
for i in range(l):
    print(f"test_{i}.start = 0")

test_0.start = 0
test_1.start = 0
test_2.start = 0
test_3.start = 0
test_4.start = 0
test_5.start = 0
test_6.start = 0
test_7.start = 0
test_8.start = 0
test_9.start = 0
test_10.start = 0
test_11.start = 0
test_12.start = 0
test_13.start = 0
test_14.start = 0
test_15.start = 0
test_16.start = 0
test_17.start = 0
test_18.start = 0
test_19.start = 0
test_20.start = 0
test_21.start = 0
test_22.start = 0
test_23.start = 0
test_24.start = 0
test_25.start = 0
test_26.start = 0
test_27.start = 0
test_28.start = 0
test_29.start = 0
test_30.start = 0
test_31.start = 0
test_32.start = 0
test_33.start = 0
test_34.start = 0
test_35.start = 0
test_36.start = 0


Generate the states and state transitions for the alu_tester_auto:

In [115]:
l = len(test_cases)
for i in range(l-1):
    print(state_case(i,False))
    print()

print(state_case(l-1, True))

            AutoStates.TEST_0:
                io_segment = test_0.io_segment
                io_select = test_0.io_select
                alucode_led = test_0.alucode_led
                led = test_0.led
                test_0.start = 1
                if(test_0.done)
                    states.d = AutoStates.TEST_1

            AutoStates.TEST_1:
                io_segment = test_1.io_segment
                io_select = test_1.io_select
                alucode_led = test_1.alucode_led
                led = test_1.led
                test_1.start = 1
                if(test_1.done)
                    states.d = AutoStates.TEST_2

            AutoStates.TEST_2:
                io_segment = test_2.io_segment
                io_select = test_2.io_select
                alucode_led = test_2.alucode_led
                led = test_2.led
                test_2.start = 1
                if(test_2.done)
                    states.d = AutoStates.TEST_3

            AutoStates.TEST_3:
         